In [12]:
import pandas as pd

# Step 1: Load the data
df = pd.read_csv('Hourly_cleaned_Patna.csv')  # Replace with your actual filename

# Step 2: Add hour index (0 to 23 repeated)
df['HourIndex'] = df.groupby(['Year', 'Month', 'Day']).cumcount() + 1

# Step 3: Pivot to reshape data so hourly values are across columns
pivoted = df.pivot_table(index=['Year', 'Month', 'Day'], 
                         columns='HourIndex', 
                         values='hourly', 
                         aggfunc='first')

# Step 4: Rename columns to HRF01 to HRF24
pivoted.columns = [f'HRF{str(i).zfill(2)}' for i in pivoted.columns]

# Step 5: Reset index to flatten the DataFrame
result = pivoted.reset_index()

# Step 6: Add INDEX, DT, DAY (day of week), TOTRF, TOT_Cal
result['INDEX'] = 42807
result['DT'] = result['Day']
result['DAY'] = pd.to_datetime(result[['Year', 'Month', 'Day']]).dt.dayofweek + 1

# Calculate TOTRF as sum of HRF01 to HRF24
result['TOTRF'] = result[[col for col in result.columns if col.startswith('HRF')]].sum(axis=1)

# Add TOT_Cal as a duplicate of TOTRF
result['TOT_Cal'] = result['TOTRF']

# Step 7: Reorder columns
ordered_cols = ['INDEX', 'Year', 'Month', 'DT', 'DAY'] + \
               [f'HRF{str(i).zfill(2)}' for i in range(1, 25)] + ['TOTRF', 'TOT_Cal']
result = result[ordered_cols]

In [13]:
result

,INDEX,Year,Month,DT,DAY,HRF01,HRF02,HRF03,HRF04,HRF05,...,HRF17,HRF18,HRF19,HRF20,HRF21,HRF22,HRF23,HRF24,TOTRF,TOT_Cal
0,42807,1969,1,1,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,42807,1969,1,2,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,42807,1969,1,3,5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,42807,1969,1,4,6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,42807,1969,1,5,7,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18988,42807,2020,12,27,7,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18989,42807,2020,12,28,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18990,42807,2020,12,29,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18991,42807,2020,12,30,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
result.to_excel('C:\\Users\\swaga\\OneDrive\\Desktop\\Patna.xlsx', index=False)

In [15]:
# Split data into training and analysis
city_analysis = result[result['Year'].between(2001, 2020)]
city_train = result[result['Year'].between(1969, 2000)]

# Create city_test using only selected columns from city_analysis
meta_columns = ['INDEX', 'Year', 'Month', 'DT', 'DAY', 'TOTRF', 'TOT_Cal']
city_test = city_analysis[meta_columns]

In [16]:
# Save to separate Excel files
city_analysis.to_csv('C:\\Users\\swaga\\OneDrive\\Desktop\\Analysis_Patna.csv', index=False)
city_train.to_csv('C:\\Users\\swaga\\OneDrive\\Desktop\\Train_Patna.csv', index=False)
city_test.to_csv('C:\\Users\\swaga\\OneDrive\\Desktop\\Test_Patna.csv', index=False)